### Snowflake connection

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
url = URL(
    account = 'fcx.west-us-2.azure',
    user = 'mcomba@fmi.com',
    database = 'PROD_WG' ,
    schema = 'MIS',
    warehouse= 'WH_ADHOC_USER',
    role = 'FCX_DATA_ANALYST_MIS',
    authenticator='externalbrowser',
)
engine = create_engine(url)
connection = engine.connect()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [2]:
query = '''SELECT TOP 500000 request_no, request_desc, request_note, justification, ticket_status,self_service_flag,request_type,prblm_item_id

FROM CC_REQUEST

WHERE self_service_flag = 'Y' AND ticket_status = 'Closed' AND utc_finish_ts > '2019-01-01'

ORDER BY request_id desc'''

data = pd.read_sql(query, connection)

In [3]:
data

,request_no,request_desc,request_note,justification,ticket_status,self_service_flag,request_type,prblm_item_id
0,PHX10870427,EOL Equipment.,EOL Equipment.,EOL Equipment.,Closed,Y,HD,17143
1,PHX10870293,Instalación de Google Chrome computadora monit...,Solicito instalar Google Chrome en la computad...,Se necesita para utilizar un add-in de excel p...,Closed,Y,HD,32936
2,PHX10870245,ELPL06556 had been decommissioned last year an...,ELPL06556 had been decommissioned last year an...,Laptop is failing now and is past EoL.,Closed,Y,HD,17143
3,PHX10870239,Please add to MDM Group for FXCMobile for Tabl...,Please add me to the Mobile Device Management ...,,Closed,Y,HD,18654
4,PHX10870235,SIE-SEC-Secondary Lockers \ 10.166.18.106 off ...,How many cameras are currently unavailable?: 1...,,Closed,Y,HD,29665
...,...,...,...,...,...,...,...,...
499995,PHX09806868,Install Adobe Acrobat Pro 2017,Software Name with Version - Adobe Acrobat Pro...,I am working on a long term project and need t...,Closed,Y,HD,30083
499996,PHX09806864,Please update the mfa phone number of account ...,Please update the mfa phone number of account ...,,Closed,Y,HD,14700
499997,PHX09806863,Configured clients new iPhone with intunes,Configured clients new iPhone with intunes,,Closed,Y,HD,18656
499998,PHX09806861,19Mo / 19Cu - WebEx and Presentation,Requesting assistance for Andrea to confirm Au...,,Closed,Y,HD,29790


In [4]:
CIT = pd.read_csv('CIT.csv', encoding = 'ansi')
CIT.head()

,ItemNo,Category,Item,Type,CIT,CIT Group,Status
0,5844,Application,Supplier Portal,Problem,Application - Supplier Portal - Problem,Problem,Inactive
1,2252,Application,Assessment and Survey System,Problem,Application - Assessment and Survey System - P...,Problem,Inactive
2,2803,Application,Ellipse/Documentos,Problem,Application - Ellipse/Documentos - Problem,Problem,Inactive
3,4737,Application,Ellipse-AP,Problem,Application - Ellipse-AP - Problem,Problem,Inactive
4,2754,Application,"EAARS - Expenditure, Acquisition & Requisition...",Problem,"Application - EAARS - Expenditure, Acquisition...",Problem,Inactive


In [5]:
CIT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3386 entries, 0 to 3385
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ItemNo     3386 non-null   int64 
 1   Category   3386 non-null   object
 2   Item       3386 non-null   object
 3   Type       3386 non-null   object
 4   CIT        3386 non-null   object
 5   CIT Group  3386 non-null   object
 6   Status     3386 non-null   object
dtypes: int64(1), object(6)
memory usage: 185.3+ KB


In [6]:
CIT.describe(include='all')

,ItemNo,Category,Item,Type,CIT,CIT Group,Status
count,3386.000000,3386,3386,3386,3386,3386,3386
unique,NaN,32,714,184,3245,4,2
top,NaN,Application,Other,Problem,Facilities - Data Center Room - Maintenance,Request,Inactive
freq,NaN,949,64,680,3,2400,2549
mean,10890.240402,NaN,NaN,NaN,NaN,NaN,NaN
std,9963.313790,NaN,NaN,NaN,NaN,NaN,NaN
min,2174.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,3700.250000,NaN,NaN,NaN,NaN,NaN,NaN
50%,4967.500000,NaN,NaN,NaN,NaN,NaN,NaN
75%,16749.750000,NaN,NaN,NaN,NaN,NaN,NaN


Change column name to merge

In [7]:
CIT = CIT.rename(columns = {'ItemNo':'prblm_item_id'})
CIT.head(1)

,prblm_item_id,Category,Item,Type,CIT,CIT Group,Status
0,5844,Application,Supplier Portal,Problem,Application - Supplier Portal - Problem,Problem,Inactive


Merge cc_request and CIT

In [8]:
df_merged = data.merge(CIT, how='left', on='prblm_item_id')
df_merged.head()

,request_no,request_desc,request_note,justification,ticket_status,self_service_flag,request_type,prblm_item_id,Category,Item,Type,CIT,CIT Group,Status
0,PHX10870427,EOL Equipment.,EOL Equipment.,EOL Equipment.,Closed,Y,HD,17143,Deskside,MIS Tech Only,Assets Disposal Request,Deskside - MIS Tech Only - Assets Disposal Req...,Request,Active
1,PHX10870293,Instalación de Google Chrome computadora monit...,Solicito instalar Google Chrome en la computad...,Se necesita para utilizar un add-in de excel p...,Closed,Y,HD,32936,Deskside,Approved Software,Auto Install,Deskside - Approved Software - Auto Install,Request,Active
2,PHX10870245,ELPL06556 had been decommissioned last year an...,ELPL06556 had been decommissioned last year an...,Laptop is failing now and is past EoL.,Closed,Y,HD,17143,Deskside,MIS Tech Only,Assets Disposal Request,Deskside - MIS Tech Only - Assets Disposal Req...,Request,Active
3,PHX10870239,Please add to MDM Group for FXCMobile for Tabl...,Please add me to the Mobile Device Management ...,,Closed,Y,HD,18654,Deskside,Mobile Device,Enroll FCX Device,Deskside - Mobile Device - Enroll FCX Device,Request,Active
4,PHX10870235,SIE-SEC-Secondary Lockers \ 10.166.18.106 off ...,How many cameras are currently unavailable?: 1...,,Closed,Y,HD,29665,Application,Security/Operational Video,Problem,Application - Security/Operational Video - Pro...,Problem,Active


Filter just problem and request CIT Group

In [9]:
df_final = df_merged[(df_merged['CIT Group'] == 'Request') | (df_merged['CIT Group'] == 'Problem')]
df_final.head()

,request_no,request_desc,request_note,justification,ticket_status,self_service_flag,request_type,prblm_item_id,Category,Item,Type,CIT,CIT Group,Status
0,PHX10870427,EOL Equipment.,EOL Equipment.,EOL Equipment.,Closed,Y,HD,17143,Deskside,MIS Tech Only,Assets Disposal Request,Deskside - MIS Tech Only - Assets Disposal Req...,Request,Active
1,PHX10870293,Instalación de Google Chrome computadora monit...,Solicito instalar Google Chrome en la computad...,Se necesita para utilizar un add-in de excel p...,Closed,Y,HD,32936,Deskside,Approved Software,Auto Install,Deskside - Approved Software - Auto Install,Request,Active
2,PHX10870245,ELPL06556 had been decommissioned last year an...,ELPL06556 had been decommissioned last year an...,Laptop is failing now and is past EoL.,Closed,Y,HD,17143,Deskside,MIS Tech Only,Assets Disposal Request,Deskside - MIS Tech Only - Assets Disposal Req...,Request,Active
3,PHX10870239,Please add to MDM Group for FXCMobile for Tabl...,Please add me to the Mobile Device Management ...,,Closed,Y,HD,18654,Deskside,Mobile Device,Enroll FCX Device,Deskside - Mobile Device - Enroll FCX Device,Request,Active
4,PHX10870235,SIE-SEC-Secondary Lockers \ 10.166.18.106 off ...,How many cameras are currently unavailable?: 1...,,Closed,Y,HD,29665,Application,Security/Operational Video,Problem,Application - Security/Operational Video - Pro...,Problem,Active


In [10]:
df_final['concat'] = df_final['request_desc']  + ' ' + df_final['request_note']  + ' ' + df_final['justification']
df_final.head()

C:\Users\Mateio\Anaconda3\envs\PyCaret\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,request_no,request_desc,request_note,justification,ticket_status,self_service_flag,request_type,prblm_item_id,Category,Item,Type,CIT,CIT Group,Status,concat
0,PHX10870427,EOL Equipment.,EOL Equipment.,EOL Equipment.,Closed,Y,HD,17143,Deskside,MIS Tech Only,Assets Disposal Request,Deskside - MIS Tech Only - Assets Disposal Req...,Request,Active,EOL Equipment. EOL Equipment. EOL Equipment.
1,PHX10870293,Instalación de Google Chrome computadora monit...,Solicito instalar Google Chrome en la computad...,Se necesita para utilizar un add-in de excel p...,Closed,Y,HD,32936,Deskside,Approved Software,Auto Install,Deskside - Approved Software - Auto Install,Request,Active,Instalación de Google Chrome computadora monit...
2,PHX10870245,ELPL06556 had been decommissioned last year an...,ELPL06556 had been decommissioned last year an...,Laptop is failing now and is past EoL.,Closed,Y,HD,17143,Deskside,MIS Tech Only,Assets Disposal Request,Deskside - MIS Tech Only - Assets Disposal Req...,Request,Active,ELPL06556 had been decommissioned last year an...
3,PHX10870239,Please add to MDM Group for FXCMobile for Tabl...,Please add me to the Mobile Device Management ...,,Closed,Y,HD,18654,Deskside,Mobile Device,Enroll FCX Device,Deskside - Mobile Device - Enroll FCX Device,Request,Active,Please add to MDM Group for FXCMobile for Tabl...
4,PHX10870235,SIE-SEC-Secondary Lockers \ 10.166.18.106 off ...,How many cameras are currently unavailable?: 1...,,Closed,Y,HD,29665,Application,Security/Operational Video,Problem,Application - Security/Operational Video - Pro...,Problem,Active,SIE-SEC-Secondary Lockers \ 10.166.18.106 off ...


In [39]:
df = df_final[df_final['Status'] == 'Active'][['concat', 'Category', 'Type','Item','Status','CIT']]
df.head()

,concat,Category,Type,Item,Status,CIT
0,EOL Equipment. EOL Equipment. EOL Equipment.,Deskside,Assets Disposal Request,MIS Tech Only,Active,Deskside - MIS Tech Only - Assets Disposal Req...
1,Instalación de Google Chrome computadora monit...,Deskside,Auto Install,Approved Software,Active,Deskside - Approved Software - Auto Install
2,ELPL06556 had been decommissioned last year an...,Deskside,Assets Disposal Request,MIS Tech Only,Active,Deskside - MIS Tech Only - Assets Disposal Req...
3,Please add to MDM Group for FXCMobile for Tabl...,Deskside,Enroll FCX Device,Mobile Device,Active,Deskside - Mobile Device - Enroll FCX Device
4,SIE-SEC-Secondary Lockers \ 10.166.18.106 off ...,Application,Problem,Security/Operational Video,Active,Application - Security/Operational Video - Pro...


In [37]:
df.groupby(['Category','Item','Type','CIT','Status','concat']).count()

Empty DataFrame
Columns: []
Index: [(Access Management, Acquire, New, Access Management - Acquire - New, Active, AcQuire database access Site: Tucson Technology center
Database: BAGACQUIREDBS SIEACQUIREDBS and MORACQUIREDBS
Server: explacquiredbs
Level of access needed: User
Justification: changing job duties, and I need to be able to log using acQuire for mine sites. Need to log core from mine site.), (Access Management, Acquire, New, Access Management - Acquire - New, Active, Acceso Base de Datos - Taladros Geología Servidor: CVEACQUIREDBS
Database: acq_fmi_cerroverde_expl
Role: User Se requiere acceso a la base de datos de taladros en AcQuire para trabajos de planificación y revisión de información referente a las actividades hidrogeológicas desarrolladas en la mina.), (Access Management, Acquire, New, Access Management - Acquire - New, Active, Acceso DBO a Acquire Solicito acceso temporal nivel DBO a acquire
"acq_fmi_cerroverde_expl"

Se requiere acceso DBO a acquire para completar el proyecto de creación de tablas y estructura de base de datos para hidrogeología.
El acceso DBO es solo temporal (3 meses), hasta culminar el proyecto indicado.
Actualmente solo tengo acceso nivel DBU

Gracias anticipadas por su atención Se requiere acceso DBO a acquire para completar el proyecto de creación de tablas y estructura de base de datos para hidrogeología.
El acceso DBO es solo temporal (3 meses), hasta culminar el proyecto indicado.
Actualmente solo tengo acceso nivel DBU), (Access Management, Acquire, New, Access Management - Acquire - New, Active, Acceso DBU a la bas de datos acq_fmi_cerroverde_orec en CVEACQUIREDBS. This request is to modify access to a database. Please provide the following:
Database: acq_fmi_cerroverde_orec server: CVEACQUIREDBS
AD Group Name: SG-CVE-SQL-ACQ-OREC-DBU
Access Type - Read/ Write - Ability to modify or delete files 
Preferred Call back Number - (+51-987303064) Se solicita permiso de lectura- escritura a la bas de datos acq_fmi_cerroverde_orec en CVEACQUIREDBS, para actualización de leyes diarias de Blast Hole, para reportes e informes.
Required for update of daily laws of Blast Hole, for reports.), (Access Management, Acquire, New, Access Management - Acquire - New, Active, Acceso a Información de blasthole Servidor: CVEACQUIREDBS
Database: acq_fmi_cerroverde_orec
Role: Read Only Estimado, 

Requiero visualizar pozos de voladura - para realizar el soporte hidrogeológico en los casos que fueran necesarios.

Saludos), (Access Management, Acquire, New, Access Management - Acquire - New, Active, Acceso a base de datos de Taladros Acceso a taladros de geología

Servidor: CVEACQUIREDBS
Database: acq_fmi_cerroverde_expl

Role: Read Only Se requiere visualizar taladros para trabajos de hidrogeología, drenes horizontales y piezómetros), (Access Management, Acquire, New, Access Management - Acquire - New, Active, Acceso a base de datos de taladros geológicos Acceso a base de datos de taladros geológicos 
Servidor: CVEACQUIREDBS base de datos: acq_fmi_cerroverde_expl
tipo: only read Acceso a base de datos de taladros geológicos para trabajos de Hidrogeología), (Access Management, Acquire, New, Access Management - Acquire - New, Active, Acceso a data base acq_fmi_cerroverde_orec Tipo de acceso: DBU
Server: CVEACQUIREDBS
DataBase: acq_fmi_cerroverde_orec Trabajos diarios de planeamiento corto plazo.), (Access Management, Acquire, New, Access Management - Acquire - New, Active, Access needed to the Acquire Production database, acq_fmi_sierrita_prod (server SIEACQUIREDBS) Richard will need access to the Acquire Production database, acq_fmi_sierrita_prod (server SIEACQUIREDBS) Richard will need access to the Acquire Production database, acq_fmi_sierrita_prod (server SIEACQUIREDBS) to perform daily ore control duties), (Access Management, Acquire, New, Access Management - Acquire - New, Active, Access to Acquire DBs Need read/write access to UGGEO_HAZARD on AcquireDBS Server. Needed for acquire data entry.), (Acce

In [40]:
df.to_csv('dataItem22')